1. База данных для хранения исходных данных, промежуточных и окончательных результатов.
Создано в DB Browser for SQLite:

CREATE TABLE "Sentences" (
	"ID"	INTEGER,
	"Sentence"	TEXT,
	"Metadata"	TEXT,
	PRIMARY KEY("ID" AUTOINCREMENT)
)
CREATE TABLE "Words" (
	"ID"	INTEGER,
	"Token"	TEXT,
	"Lemma"	TEXT,
	"POS"	TEXT,
	"ID_sent"	INTEGER,
	PRIMARY KEY("ID" AUTOINCREMENT)
)

In [100]:
import sqlite3
from conllu import parse
from os import listdir

In [98]:
conn = sqlite3.connect('DoctorWho.db')
cur = conn.cursor()

Для коллекции предложений conllu идёт первичная итерация по предложениям sent с метадатой meta, вторичная итерация по токенам token с леммой lemma и тегом tag

In [104]:
# пишем данные в базу
id_sent = 1
id_token = 1
for file in listdir('fics_conllus/conllu'):
    filename = 'fics_conllus/conllu/' + file
    with open(filename) as f:
        text = f.read()
    tokens = parse(text)
    for line in tokens:
        meta = 'test'
        sent = str(line.metadata['text'])
        cur.execute("INSERT INTO Sentences VALUES (?,?,?)", (id_sent, sent, meta)) # id, предложение, метадата
        for word in line:
            token = word['form'].lower()
            lemma = word['lemma']
            tag = word['upos']
            if tag != 'PUNCT':
                cur.execute(
                    """
                        INSERT INTO Words
                        VALUES (?,?,?,?,?)
                    """, (id_token,token,lemma,tag,id_sent)) # id, токен, лемма, тег, id предложения, из которго взято слово
                id_token +=1
        id_sent += 1
conn.commit()

In [81]:
# request = [([entry], search_type)], search_type IN token, lemma, POS, (lemma,POS)
# Если нужна последовательность из 2-3 слов
# Вынимаем из базы  ID предложений, в которых встречаются искомые вхождения и ID самих вхождений
def search_sequence(request):
    i = 1
    output = []
    for entry in request:
        if entry[1] != 'lemma+pos':
            query = 'SELECT ID_sent, ID FROM Words WHERE Words.'+ entry[1]+ '="'  + entry[0][0] +'"'
        else:
            query = 'SELECT ID_sent, ID FROM Words WHERE Words.lemma="'  + entry[0][0] + '"AND Words.POS="' + entry[0][1]+'"'
        i +=1
        res = cur.execute(query)
        output.append(res.fetchall())
    return(output)

In [5]:
# Продолжение поиска последовательности из 2-3 слов
# Из предыдущей функции мы получили для каждого запроса список пар (ID предложений, в которых встречаются искомые вхождения, ID самих вхождений)
# и сложили их в список второго уровня, из двух или трёх элементов. Собираем теперь словарь нужных нам предложений. 
def merge_sequence(one, two, first = {}):
    res = {}
    second = {}
    if not first:
        for entry in one:
            if entry[0] not in first.keys():
                first[entry[0]] = [entry[1]+1]
            else:
                first[entry[0]].append(entry[1]+1)
    else:
        for entry in two:
             if entry[0] in first.keys():
                if entry[0] not in second.keys():
                     second[entry[0]] = [entry[1]]
                else:
                    second[entry[0]].append(entry[1])
        for key in first.keys():
            for value in first[key]:
                value += 1
    for key in first.keys():
        if set(first[key]) & set(second[key]):
            res[key] = set(first[key]) & set(second[key])
    return(res)

In [66]:
# request = [([entry], search_type)], search_type IN token, lemma, POS, (lemma,POS)
# Если нужно одно слово - сразу вынимаем из базы предложения и метадату
def search_one(request):
    if request[0][1] != 'lemma+pos':
        query = 'SELECT Sentence, Metadata FROM Sentences JOIN Words ON Words.ID_sent = Sentences.ID WHERE Words.' + request[0][1]+ '="' + request[0][0][0]+'"'
    else:
        query = 'SELECT Sentence, Metadata FROM Sentences JOIN Words ON Words.ID_sent = Sentences.ID WHERE Words.lemma' + '="' + request[0][0][0]+ '" AND Words.POS' + '="' + request[0][0][1] + '"'
    res = cur.execute(query)
    result = res.fetchall()
    return(result)

In [83]:
# test of functions
# search one
lemma_search = [(['парень'], 'lemma')]
token_search = [(['парня'], 'token')]
pos_search = [(['NOUN'], 'POS')]
lemma_pos_search = [(['парень','NOUN'], 'lemma+pos')]
# sequence search
long_search = [(['парень'], 'lemma'), (['парень','NOUN'], 'lemma+pos')]

In [96]:
long_search[0][1]

'lemma'

In [97]:
search_sequence(long_search)

[[(1, 5), (5, 43), (12, 106), (19, 152), (30, 286), (37, 349)],
 [(1, 5), (5, 43), (12, 106), (19, 152), (30, 286), (37, 349)]]

In [39]:
# request = [([entry], search_type)], search_type IN token, lemma, POS, (lemma,POS)
request = lemma_search
if len(request) == 1:
    output = search_one(request)
    print(output)
else:
    output = search_sequence(request)
    l = len(output)
    flag = True
    for i in range(l-1):
        if flag:
            result = merge_sequence(output[i], output[i+1])
            flag = False
        else:
            result = merge_sequence(output[i], output[i+1], result)
    if result:
        output = cur.executemany("SELECT Sentence, Metadata FROM Sentences JOIN Words ON Words.ID_sent = Sentences.ID WHERE Words.ID = ?", result.keys())
        output.fetchall()
    else:
        output = "Sorry, no matches to your request"

OperationalError: no such column: парень

In [93]:
conn.close()